# Creating a basic Cluster Expansion

In [2]:
import random
import numpy as np
from monty.serialization import loadfn, dumpfn
from pymatgen.core.structure import Structure
from smol.cofe import ClusterSubspace, StructureWrangler, ClusterExpansion, RegressionData

No module named 'phonopy'


In [3]:
# load the prim structure
#prim_path = '/Users/myless/Dropbox (MIT)/Research/2024/Spring_2024/Computation/structure_maker/v4cr4ti_prim_cell.json'
prim_path = '/home/myless/Packages/structure_maker/vcrtiwzr_prim_entry.json'
lno_prim = loadfn(prim_path)
    
# load the fitting data
#entry_path = '/Users/myless/Dropbox (MIT)/Research/2024/Spring_2024/Computation/structure_maker/vcrti_fixed_entries.json'
entry_path = '/home/myless/Packages/structure_maker/vcrtiwzr_cse.json'
lno_entries = loadfn(entry_path)

### 0) The prim structure
The prim structure defines the **configurational space** for the Cluster Expansion. 
The **configurational space** is defined by the site **compositional spaces** and the crystal symetries of the prim structure.
The occupancy of the sites determine site **compositional spaces**. Sites are **active** if they have compositional degrees of freedom.


Active sites have fractional compositions. Vacancies are allowed in sites where the composition does not sum to one.

0. Is active. The allowed species are: Li+ and vacancies.
1. Is active. The allowed species are: Ni3+ and Ni4+.
2. Is not active. Only O2- is allowed.
3. Is not active. Only O2- is allowed.

In [4]:
print(lno_prim)

Full Formula (Zr0.02 Ti0.04 V0.18 Cr0.04 W0.08)
Reduced Formula: Zr0.02Ti0.04V0.18Cr0.04W0.08
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.010, Ti:0.020, V:0.090, Cr:0.020, W:0.040  0    0    0
  1  Zr:0.010, Ti:0.020, V:0.090, Cr:0.020, W:0.040  0.5  0.5  0.5


### 1) The cluster subspace
The `ClusterSubspace` represents all the orbits (groups of equivalent clusters) that will be considered when fitting the cluster expansion. Its main purpose is to compute the **correlations functions** for each included orbit given a structure in the compositional space defined by the prim.

In order to do be able to compute the correlation functions, the given structure must match the prim structure in a "crystallographic" sense but allowing for compositional degrees of freedom in the "active" sites.

A cluster subspace most easily created by providing:
1. The prim structure representing the configurational space.
2. A set of diameter cutoffs for each size of orbit we want to consider.
3. A type of site basis function to use.

There are more options allowed by the code to fine grain and tune. See other notebooks for advanced use cases.

In [5]:
subspace = ClusterSubspace.from_cutoffs(
    lno_prim,
    cutoffs={2: 5, 3: 4.1}, # will include orbits of 2 and 3 sites.
    basis='sinusoid', # sets the site basis type, default is indicator
    supercell_size='num_sites'
)

# supercell_size specifies the method to determine the supercell size
# when trying to match a structure.
# (See pymatgen.structure_matcher.StructureMatcher for more info)

print(subspace) # single site and empty orbits are always included.

Basis/Orthogonal/Orthonormal : sinusoid/True/False
       Unit Cell Composition : V0.18 Cr0.04 Ti0.04 Zr0.02 W0.08
            Number of Orbits : 7
No. of Correlation Functions : 141
             Cluster Cutoffs : 2: 4.99, 3: 3.01
              External Terms : []
Orbit Summary
 ------------------------------------------------------------------------
 |  ID     Degree    Cluster Diameter    Multiplicity    No. Functions  |
 |   0       0             NA                 0                1        |
 |   1       1            0.0000              2                5        |
 |   2       2            2.6067              8               15        |
 |   3       2            3.0100              6               15        |
 |   4       2            4.2568              12              15        |
 |   5       2            4.9915              24              15        |
 |   6       3            3.0100              24              75        |
 ------------------------------------------------------

#### 1.1) Computing a correlation vector.
A correlation vector for a specific structure (represents the feature vector) used to train and predict target values.

In [6]:
structure = lno_entries[1].structure
corr = subspace.corr_from_structure(structure)

print(f'The correlation vector for a structure with'
      f' composition {structure.composition} is: '
      f'\n{corr}')

The correlation vector for a structure with composition Zr2 Ti12 V38 Cr8 W4 is: 
[ 1.          0.328125   -0.62245576  0.265625    0.29769623 -0.375
  0.06835937 -0.21312344  0.09667969  0.0727326  -0.11328125  0.39257812
 -0.18436869 -0.20214844  0.26386712  0.0859375   0.09810444 -0.10546875
  0.08203125 -0.115019    0.109375    0.14192708 -0.19395361  0.08203125
  0.13080592 -0.15885417  0.38671875 -0.16237976 -0.16796875  0.2119958
  0.07421875  0.08118988 -0.1171875   0.10546875 -0.13982702  0.20833333
  0.11132812 -0.21086816  0.0859375   0.11952955 -0.14453125  0.38085938
 -0.16914559 -0.1796875   0.23229327  0.06835938  0.07780697 -0.08984375
  0.11132812 -0.13757174  0.15625     0.09440104 -0.20635762  0.09277344
  0.07047733 -0.09114583  0.38476562 -0.1573054  -0.18652344  0.230038
  0.0625      0.08457279 -0.109375    0.04296875 -0.06314769  0.10416667
  0.01969401 -0.04707885  0.02832031  0.03044621 -0.04817708 -0.08880143
  0.12841797 -0.06258387 -0.08105469  0.10487026  0

### 2) The structure wrangler
The `StructureWrangler` is a class that will is used to create and organize the data that will be used to train (and possibly test) the cluster expansion. It makes sure that all the supplied structures appropriately match the prim structure, and obtains the necessary information to correctly normalize target properties (such as energy) necessary for training.

Training data is added to a `StructureWrangler` using `ComputedStructureEntry` instances from `pymatgen`.

Matching relaxed structures can be a tricky problem, especially for ionic systems with vacancies. See the notebook on structure matching for tips on how to tweak parameters.

In [7]:
wrangler = StructureWrangler(subspace)

# the energy is taken directly from the ComputedStructureEntry
# any additional properties can also be added, see notebook on
# training data preparation for an example.
for entry in lno_entries:
    wrangler.add_entry(entry, verbose=True)
# The verbose flag will print structures that fail to match.

print(f'\nTotal structures that match {wrangler.num_structures}/{len(lno_entries)}')


Total structures that match 60/60


## 3) Training

Training a cluster expansion is one of the most critical steps. This is how you get **effective cluster interactions (ECI's)**. To do so you need an estimator class that implements some form of regression model. In this case we will use simple least squares regression using the `LinearRegression` estimator from `scikit-learn`.

In `smol` the coefficients from the fit are not exactly the ECI's but the ECI times the multiplicity of their orbit.

In [8]:
from sklearn.linear_model import LinearRegression
# Set fit_intercept to False because we already do this using
# the empty cluster.
estimator = LinearRegression(fit_intercept=False)
estimator.fit(wrangler.feature_matrix, wrangler.get_property_vector('energy'))
coefs = estimator.coef_

#### 3.1) Check the quality of the fit
There are many ways to evaluate the quality of a fit. The simplest involve stadard training set prediction error metrics. But when evaluating a CE more seriously we need to consider further metrics and how the CE will be used.
Here we will just look at in sample mean squared error and max error.

In [9]:
from sklearn.metrics import mean_squared_error, max_error

train_predictions = np.dot(wrangler.feature_matrix, coefs)

rmse = mean_squared_error(
    wrangler.get_property_vector('energy'), train_predictions, squared=False
)
maxer = max_error(wrangler.get_property_vector('energy'), train_predictions)

print(f'RMSE {1E3 * rmse} meV/prim')
print(f'MAX {1E3 * maxer} meV/prim')

RMSE 1.5337755261224768e-10 meV/prim
MAX 2.4513724383723456e-10 meV/prim


/home/myless/.mambaforge/envs/chgnet/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 4) The cluster expansion
Now we can use the above work to create the `ClusterExpansion`. The cluster expansion can be used to predict the fitted property for new structures, either for testing quality or for simulations such as in Monte Carlo.
Note that when using the `predict` function, the cluster expansion will have to match the given structure if it has not seen it before.
We will also store the details of the regression model used to fit the cluster expansion by using a `RegressionData` object.

In [10]:
reg_data = RegressionData.from_sklearn(
    estimator, wrangler.feature_matrix,
    wrangler.get_property_vector('energy')
)


expansion = ClusterExpansion(
    subspace, coefficients=coefs, regression_data=reg_data
)

structure = random.choice(wrangler.structures)
prediction = expansion.predict(structure, normalized=True)

print(
    f'The predicted energy for a structure with composition '
    f'{structure.composition} is {prediction} eV/prim.\n'
)
print(f'The fitted coefficients are:\n{expansion.coefs}\n')
print(f'The effective cluster interactions are:\n{expansion.eci}\n')
print(expansion)

The predicted energy for a structure with composition Zr1 Ti6 V43 Cr12 W2 is -18.098820106874907 eV/prim.

The fitted coefficients are:
[-1.54061733e+01 -3.35239653e+00  2.68870090e+00  7.64057383e-01
  2.54995371e+00  2.21610340e+00 -3.92503947e-01  1.77901138e-01
  2.72917831e-01  1.03976704e+00  5.77911601e-01 -1.05802981e-01
 -4.32042268e-02 -1.22292834e+00 -1.66165375e-01 -4.09695813e-01
  3.11891844e-01 -5.46824343e-01  3.17960454e-01 -2.54628155e-01
 -3.62531686e-01 -6.91274469e-01  8.80992978e-01  2.26932351e-01
  5.06652493e-01  3.07172479e-02 -2.30428914e-01 -2.48330858e-01
 -6.69306797e-01 -9.89102791e-04  7.36503531e-03  3.35647032e-01
 -6.95726118e-01 -3.48901535e-01 -6.15359688e-01 -4.22112617e-01
 -7.49276829e-01  8.78541602e-01 -5.40903641e-03  4.87752739e-01
  2.42037518e-01  4.53338379e-01  1.13646855e-02 -9.40775507e-01
 -9.77258115e-01 -1.10951436e-01 -2.45490764e-01 -2.52996721e-01
 -3.63425591e-02 -1.24292074e-01  1.86871730e-01 -4.38031296e-02
  2.88112922e-01  1

### 5) Saving your work
All core classes in `smol` are `MSONables` and so can be saved using their `as_dict` methods or better yet with `monty.serialization.dumpfn`.

Currently there is also a convenience function in `smol` that will nicely save all of your work for you in a standardized way. Work saved with the `save_work` function is saved as a dictionary with standardized names for the classes. Since a work flow should only contain 1 of each core classes the function will complain if you give it two of the same class (i.e. two wranglers)

In [11]:
from smol.io import save_work

#file_path = 'v4cr4ti_fin_work.mson'
file_path = 'v2cr2ti1zr4w_fin_work.mson'
# we can save the subspace as well, but since both the wrangler
# and the expansion have it, there is no need to do so.
save_work(file_path, wrangler, expansion)

#### 5.1) Loading previously saved work

In [12]:
from smol.io import load_work

work = load_work(file_path)
for name, obj in work.items():
    print(f'{name}: {type(obj)}\n')

StructureWrangler: <class 'smol.cofe.wrangling.wrangler.StructureWrangler'>

ClusterExpansion: <class 'smol.cofe.expansion.ClusterExpansion'>

